# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 13 2023 8:30AM,254963,10,0086296211,ISDIN Corporation,Released
1,Jan 13 2023 7:54AM,254976,19,ADV80010365,"AdvaGen Pharma, Ltd",Released
2,Jan 13 2023 7:54AM,254976,19,ADV80010366,"AdvaGen Pharma, Ltd",Released
3,Jan 13 2023 7:54AM,254976,19,ADV80010367,"AdvaGen Pharma, Ltd",Released
4,Jan 13 2023 7:54AM,254976,19,ADV80010368,"AdvaGen Pharma, Ltd",Released
5,Jan 13 2023 7:54AM,254976,19,ADV80010369,"AdvaGen Pharma, Ltd",Released
6,Jan 13 2023 7:54AM,254976,19,ADV80010370,"AdvaGen Pharma, Ltd",Released
7,Jan 13 2023 7:54AM,254976,19,ADV80010371,"AdvaGen Pharma, Ltd",Released
8,Jan 13 2023 7:54AM,254976,19,ADV80010375,"AdvaGen Pharma, Ltd",Released
9,Jan 13 2023 7:54AM,254976,19,ADV80010376,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
8,254949,Released,4
9,254961,Executing,1
10,254963,Released,1
11,254975,Released,1
12,254976,Released,48


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254949,NaN,NaN,4.0
254961,NaN,1.0,NaN
254963,NaN,NaN,1.0
254975,NaN,NaN,1.0
254976,NaN,NaN,48.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254899,0.0,0.0,1.0
254905,0.0,0.0,1.0
254908,0.0,1.0,1.0
254909,1.0,0.0,3.0
254918,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254899,0,0,1
254905,0,0,1
254908,0,1,1
254909,1,0,3
254918,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254899,0,0,1
1,254905,0,0,1
2,254908,0,1,1
3,254909,1,0,3
4,254918,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254899,,,1
1,254905,,,1
2,254908,,1,1
3,254909,1,,3
4,254918,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd"
49,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd"
50,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC"
51,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC"
55,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc."
56,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc."
57,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation
61,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc."
63,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,,,1
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd",,,48
2,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd",,,1
3,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",,1,
4,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",,,4
5,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",,1,
6,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc.",,,1
7,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,1,,3
8,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",,1,1
9,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1,,
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd",48,,
2,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd",1,,
3,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",,1,
4,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4,,
5,Jan 12 2023 12:35PM,254922,21,"NBTY Global, Inc.",,1,
6,Jan 12 2023 12:22PM,254918,21,"NBTY Global, Inc.",1,,
7,Jan 12 2023 11:52AM,254909,10,ISDIN Corporation,3,,1
8,Jan 12 2023 11:39AM,254908,19,"GUSA Granules USA, Inc.",1,1,
9,Jan 12 2023 10:48AM,254905,19,"GUSA Granules USA, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1,,
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd",48,,
2,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd",1,,
3,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",,1,
4,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1.0,NaN,NaN
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd",48.0,NaN,NaN
2,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd",1.0,NaN,NaN
3,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",NaN,1.0,NaN
4,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 13 2023 8:30AM,254963,10,ISDIN Corporation,1.0,0.0,0.0
1,Jan 13 2023 7:54AM,254976,19,"AdvaGen Pharma, Ltd",48.0,0.0,0.0
2,Jan 13 2023 7:46AM,254975,19,"AdvaGen Pharma, Ltd",1.0,0.0,0.0
3,Jan 12 2023 3:42PM,254961,12,"SD Head USA, LLC",0.0,1.0,0.0
4,Jan 12 2023 2:52PM,254949,10,"Snap Medical Industries, LLC",4.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,764821,8.0,0.0,1.0
12,254961,0.0,1.0,0.0
17,254899,1.0,0.0,0.0
19,1019764,51.0,1.0,0.0
21,509840,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,764821,8.0,0.0,1.0
1,12,254961,0.0,1.0,0.0
2,17,254899,1.0,0.0,0.0
3,19,1019764,51.0,1.0,0.0
4,21,509840,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,8.0,0.0,1.0
1,12,0.0,1.0,0.0
2,17,1.0,0.0,0.0
3,19,51.0,1.0,0.0
4,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,8.0
1,12,Released,0.0
2,17,Released,1.0
3,19,Released,51.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,17,19,21
Status,,,,,
Completed,1.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,0.0,1.0,1.0
Released,8.0,0.0,1.0,51.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,17,19,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,1.0,1.0
2,Released,8.0,0.0,1.0,51.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,17,19,21
0,Completed,1.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,0.0,1.0,1.0
2,Released,8.0,0.0,1.0,51.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()